In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from pathlib import Path

# --- Path Setup ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

# --- Imports from src ---
from config import CONFIG
from skeleton_defs import SKELETON_HIERARCHY 

# --- External Metadata Integration (Subject Data) ---
# Path to the external JSON file containing anthropometric data
METADATA_PATH = os.path.join(PROJECT_ROOT, "data", "subject_metadata.json")

def load_subject_metadata(path):
    """
    Loads subject-specific data (weight, height) from a JSON file.
    If file is missing or values are null, the pipeline switches to 
    'Relative Normalization Mode' based on Hof (1996).
    """
    if os.path.exists(path):
        with open(path, 'r') as f:
            return json.load(f)
    return None

# Global Subject Variables
metadata = load_subject_metadata(METADATA_PATH)
SUBJECT_WEIGHT = None
SUBJECT_HEIGHT = None
PIPELINE_MODE = "Normalization (Unit Mass)"

if metadata and "subject_info" in metadata:
    info = metadata["subject_info"]
    SUBJECT_WEIGHT = info.get("weight_kg")
    SUBJECT_HEIGHT = info.get("height_cm")
    
    if SUBJECT_WEIGHT and SUBJECT_HEIGHT:
        PIPELINE_MODE = "Scientific (Anthropometric)"
        print(f"✅ Scientific Mode: Using Winter (2009) coefficients for {SUBJECT_WEIGHT}kg")
    else:
        print("⚠️ Subject info found but incomplete. Using Internal Normalization.")
else:
    print("ℹ️ No metadata file found. Defaulting to Relative Normalization Mode.")

# --- Directories ---
DERIV_01 = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_01_parse")
DERIV_02 = os.path.join(PROJECT_ROOT, CONFIG['derivatives_dir'], "step_02_preprocess")
QC_02 = os.path.join(PROJECT_ROOT, CONFIG['qc_dir'], "step_02_preprocess")

os.makedirs(DERIV_02, exist_ok=True)
os.makedirs(QC_02, exist_ok=True)

print(f"Ready. Mode: {PIPELINE_MODE}")
print(f"Output directory: {DERIV_02}")

⚠️ Subject info found but incomplete. Using Internal Normalization.
Ready. Mode: Normalization (Unit Mass)
Output directory: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_02_preprocess


In [2]:
# --- Data Loading and Run ID Definition ---
# Derive the parquet file from the CSV path in config.
# This ensures notebook 02 always processes the same file as notebook 01.
csv_filename = Path(CONFIG['current_csv']).stem  # Gets filename without extension
RUN_ID = csv_filename
PARQUET_PATH = Path(DERIV_01) / f"{RUN_ID}__parsed_run.parquet"

# SCIENTIFIC RATIONALE: Consistent file tracking is essential for 
# Reproducible Research in Biomechanics (Winter, 2009).
if not PARQUET_PATH.exists():
    print(f"❌ ERROR: Expected parquet file not found: {PARQUET_PATH}")
    print(f"Did you run notebook 01 first?")
    raise FileNotFoundError(f"Parquet file not found: {PARQUET_PATH}")

print(f"Loading Run ID: {RUN_ID}")
print(f"File: {PARQUET_PATH}")

# Loading the parsed data
df_raw = pd.read_parquet(PARQUET_PATH)

# ISB COMPLIANCE NOTE: Standardizing column structures at this stage 
# facilitates the mapping of global coordinate systems to segment locals 
# according to Wu et al. (2005) standards.
print(f"✅ Loaded successfully. Shape: {df_raw.shape}")

Loading Run ID: 734_T1_P1_R1_Take 2025-12-01 02.18.27 PM
File: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_01_parse\734_T1_P1_R1_Take 2025-12-01 02.18.27 PM__parsed_run.parquet
✅ Loaded successfully. Shape: (30798, 359)


In [3]:
# --- CELL 03: Data Standardization and Preprocessing ---
# RATIONALE: Standardize column names and filter to hierarchy-essential joints

from preprocessing import validate_time_vector, validate_quaternion_completeness
from skeleton_defs import SKELETON_HIERARCHY

def standardize_to_hierarchy(df_raw, hierarchy_dict):
    """
    Standardizes the raw data to match the skeleton hierarchy.
    This is the critical preprocessing step that creates df_preprocessed.
    """
    print(f"\n{'='*20} DATA STANDARDIZATION {'='*20}")
    
    # 1. Get hierarchy-essential joints
    essential_joints = set(hierarchy_dict.keys())
    available_joints = set([col.split('__')[0] for col in df_raw.columns if '__' in col])
    
    # 2. Filter to essential joints only
    essential_columns = ['frame_idx', 'time_s']  # Keep metadata columns
    for joint in essential_joints:
        if joint in available_joints:
            # Add all columns for this joint (position + quaternion)
            joint_cols = [col for col in df_raw.columns if col.startswith(f"{joint}__")]
            essential_columns.extend(joint_cols)
    
    df_standardized = df_raw[essential_columns].copy()
    
    # 3. Report
    kept_joints = set([col.split('__')[0] for col in df_standardized.columns if '__' in col])
    print(f"Original joints: {len(available_joints)}")
    print(f"Hierarchy essential: {len(essential_joints)}")
    print(f"Kept for processing: {len(kept_joints)}")
    print(f"✅ Data standardized to Hierarchy Report. Shape: {df_standardized.shape}")
    
    return df_standardized

# 1. Time Monotonicity Check
try:
    validate_time_vector(df_raw['time_s'].values)
    print("✅ Time vector validation: Monotonic")
except ValueError as e:
    print(f"❌ Time vector validation FAILED: {e}")
    raise

# 2. Standardize data to hierarchy
df_preprocessed = standardize_to_hierarchy(df_raw, SKELETON_HIERARCHY)

# 3. Quaternion Completeness Check (now on standardized data)
joint_names = set([col.split('__')[0] for col in df_preprocessed.columns if '__' in col])

missing_quat_joints = []
for joint in joint_names:
    try:
        validate_quaternion_completeness(df_preprocessed.columns, joint)
    except ValueError as e:
        missing_quat_joints.append(joint)
        print(f"❌ Quaternion validation FAILED for {joint}: {e}")

if missing_quat_joints:
    print(f"❌ {len(missing_quat_joints)} joints have incomplete quaternion data")
    print(f"⚠️  Will proceed with available joints only")
else:
    print(f"✅ Quaternion validation: All {len(joint_names)} joints complete")

print(f"✅ Schema validation passed for {len(joint_names)} joints")

✅ Time vector validation: Monotonic

==================== DATA STANDARDIZATION ====================
Original joints: 51
Hierarchy essential: 27
Kept for processing: 27
✅ Data standardized to Hierarchy Report. Shape: (30798, 191)
✅ Quaternion validation: All 27 joints complete
✅ Schema validation passed for 27 joints


In [4]:
# --- UPDATED CELL 04: Build Kinematics Map (The Scientific Blueprint) ---

def build_map_from_available_joints(df_columns, hierarchy_dict):
    """
    Scans the current DataFrame columns and builds the kinematics map.
    This ensures NB 06 and NB 08 only attempt calculations on valid joint-chains.
    """
    print(f"\n{'='*20} BUILDING KINEMATICS MAP {'='*20}")
    
    kinematics_map = {}
    
    # Identify joints that survived filtering in Cell 3
    # We look for the base name before the '__px' suffix
    existing_segments = set([c.split('__')[0] for c in df_columns if '__' in c])
    
    skipped_count = 0
    kept_count = 0
    
    for segment, info in hierarchy_dict.items():
        parent = info['parent']
        angle_name = info['angle_name']
        
        # 1. Check if joint exists in current data
        if segment not in existing_segments:
            skipped_count += 1
            continue
            
        # 2. BIOMECHANICAL INTEGRITY: Parent must exist for relative calculations
        if parent is not None and parent not in existing_segments:
            print(f"⚠️ SCIENTIFIC WARNING: Orphaned Joint '{segment}'.")
            print(f"   Cannot calculate '{angle_name}' because parent '{parent}' was filtered out.")
            continue

        # 3. Validation Passed: Add to map for downstream Notebooks
        kinematics_map[segment] = {
            "parent": parent,
            "angle_name": angle_name,
            "is_global": (parent is None)
        }
        kept_count += 1

    print(f"Total defined in Schema: {len(hierarchy_dict)}")
    print(f"Skipped (Missing/Filtered): {skipped_count}")
    print(f"Mapped (Ready for Physics):  {kept_count}")
    print(f"{'='*45}\n")
    
    return kinematics_map

# --- EXECUTE ---
# Note: Use df_preprocessed from Cell 3
kinematics_map = build_map_from_available_joints(df_preprocessed.columns, SKELETON_HIERARCHY)


==================== BUILDING KINEMATICS MAP ====================
Total defined in Schema: 27
Skipped (Missing/Filtered): 0
Mapped (Ready for Physics):  27



In [5]:
# --- CELL 05: Gap Filling and Rotational Re-normalization ---
# RATIONALE: Biomechanical analysis requires continuous derivative signals (velocity/acceleration).
# Small gaps (<100ms) are safely interpolated to prevent signal fragmentation.

# Configuration: 10 frames at 120Hz = 83.3ms. 
# Scientific Limit: Skurowski (2021) suggests avoiding interpolation for gaps > 100ms in dynamic movement.
MAX_GAP_SIZE = 10  

def fill_missing_data(df, max_gap):
    """
    1. Performs BOUNDED linear interpolation for small kinematic gaps (NO boundary extrapolation).
    2. Re-normalizes quaternions to maintain unit length (Rotational Integrity).
    """
    df_clean = df.copy()
    
    # 1. Bounded Linear Interpolation (NO boundary extrapolation)
    # Only fill gaps that are completely surrounded by valid data
    df_clean = df_clean.interpolate(method='linear', limit=max_gap, limit_area='inside')
    
    # 2. Quaternion Re-normalization
    # RATIONALE: Linear interpolation of quaternions (LERP) leads to non-unit vectors.
    # Without re-normalization, angular velocity calculations in NB 06 will be distorted.
    quat_cols = [c for c in df_clean.columns if c.endswith(('__qx', '__qy', '__qz', '__qw'))]
    segments = set(c.split('__')[0] for c in quat_cols)
    
    for seg in segments:
        try:
            qx, qy = df_clean[f"{seg}__qx"], df_clean[f"{seg}__qy"]
            qz, qw = df_clean[f"{seg}__qz"], df_clean[f"{seg}__qw"]
            
            norms = np.sqrt(qx**2 + qy**2 + qz**2 + qw**2)
            norms[norms == 0] = 1.0  # Avoid division by zero
            
            df_clean[f"{seg}__qx"] /= norms
            df_clean[f"{seg}__qy"] /= norms
            df_clean[f"{seg}__qz"] /= norms
            df_clean[f"{seg}__qw"] /= norms
        except KeyError:
            continue # Skip if a specific quaternion component is missing
            
    return df_clean

# Store pre-interpolation NaN count for audit
nan_before_basic = df_preprocessed.isna().sum().sum()
print(f"🔍 AUDIT: NaNs BEFORE basic interpolation: {nan_before_basic}")

print(f"Running Bounded Gap Filling (Max Gap: {MAX_GAP_SIZE} frames, NO boundary extrapolation)...")
# Note: df_preprocessed continues the chain from Cell 03 standard naming
df_preprocessed = fill_missing_data(df_preprocessed, MAX_GAP_SIZE)

# Store post-interpolation NaN count for audit
nan_after_basic = df_preprocessed.isna().sum().sum()
gaps_filled_basic = nan_before_basic - nan_after_basic

print(f"🔍 AUDIT: NaNs AFTER basic interpolation: {nan_after_basic}")
print(f"🔍 AUDIT: Gaps filled by basic interpolation: {gaps_filled_basic}")
print(f"✅ Bounded Gap Filling Complete.")
print(f"📊 Quality Control: Remaining NaNs (Critical Gaps > {MAX_GAP_SIZE/120}s): {nan_after_basic}")

# DETAILED POSITION MISSING STATISTICS
pos_cols = [c for c in df_preprocessed.columns if c.endswith(('__px', '__py', '__pz'))]
print(f"\n📈 Position Missing Data Analysis:")
print(f"Mean missing: {df_preprocessed[pos_cols].isna().mean().mean():.4f}")
print(f"Max missing: {df_preprocessed[pos_cols].isna().mean().max():.4f}")
print("Missing data summary:")
print(df_preprocessed[pos_cols].isna().mean().describe())

🔍 AUDIT: NaNs BEFORE basic interpolation: 0
Running Bounded Gap Filling (Max Gap: 10 frames, NO boundary extrapolation)...
🔍 AUDIT: NaNs AFTER basic interpolation: 0
🔍 AUDIT: Gaps filled by basic interpolation: 0
✅ Bounded Gap Filling Complete.
📊 Quality Control: Remaining NaNs (Critical Gaps > 0.08333333333333333s): 0

📈 Position Missing Data Analysis:
Mean missing: 0.0000
Max missing: 0.0000
Missing data summary:
count    81.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64


In [6]:
# --- CELL 05.5: Advanced Gap Filling (Optional) ---
# RATIONALE: This cell provides artifact detection and bounded spline interpolation
# for high-fidelity gap filling. It's optional and can be skipped if data is already clean.

# Import required modules
from gapfill_positions import gap_fill_positions
from artifacts import apply_artifact_truncation

# Configuration for advanced gap filling - moved to global scope
MAD_MULTIPLIER = 6.0  # Conservative threshold for artifact detection
MAX_GAP_S = 0.1      # Maximum gap duration in seconds (100ms)

def advanced_gap_filling(df, fs_target):
    """
    Apply advanced gap filling with artifact detection and bounded spline interpolation.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame with position and quaternion data
    fs_target : float
        Target sampling frequency in Hz
        
    Returns:
    --------
    df_filled : pd.DataFrame
        DataFrame with gaps filled
    """
    print("🔍 Detecting artifacts in position channels...")
    
    # Get position columns and group by joint
    pos_cols = [c for c in df.columns if c.endswith(("__px", "__py", "__pz"))]
    joints = set(c.split("__")[0] for c in pos_cols)
    
    # Apply artifact truncation to position data
    df_filled = df.copy()
    total_newly_masked = 0
    
    for joint in joints:
        # Get all position columns for this joint
        joint_cols = [f"{joint}__px", f"{joint}__py", f"{joint}__pz"]
        joint_cols = [c for c in joint_cols if c in df.columns]
        
        if len(joint_cols) == 3:  # Need all 3 axes
            try:
                # Extract position data as (N, 3) array
                position_data = df[joint_cols].values
                time_s = df["time_s"].values
                
                pos_clean, mask_raw, mask_expanded = apply_artifact_truncation(
                    position_data, time_s, 
                    mad_multiplier=MAD_MULTIPLIER, 
                    dilation_frames=1
                )
                
                # Fix: Correct newly masked calculation (cannot be negative)
                newly_masked = int(np.sum(mask_expanded & ~np.isnan(position_data)))
                total_newly_masked += newly_masked
                
                # Fix: Safe assignment - assign full column
                for i, col in enumerate(joint_cols):
                    df_filled[col] = pos_clean[:, i]
                    
            except Exception as e:
                print(f"   ⚠️  Could not process {joint}: {e}")
                continue
    
    channels_with_artifacts = 0
    for col in pos_cols:
        if col in df.columns and col in df_filled.columns:
            orig_nans = np.sum(np.isnan(df[col].values))
            filled_nans = np.sum(np.isnan(df_filled[col].values))
            if filled_nans > orig_nans:
                channels_with_artifacts += 1
                
    print(f"   ✅ Artifacts detected in {channels_with_artifacts} channels, total newly masked: {total_newly_masked} points")
    
    # Apply bounded spline gap filling (NO boundary extrapolation)
    print(f"🔧 Filling gaps (max {MAX_GAP_S}s) with bounded spline...")
    time_s = df["time_s"].values
    
    # Store pre-advanced fill NaN count for audit
    nan_before_advanced = df_filled.isna().sum().sum()
    
    try:
        df_filled = gap_fill_positions(df_filled, time_s, max_gap_s=MAX_GAP_S, min_run_length=5)
        
        # Store post-advanced fill NaN count for audit
        nan_after_advanced = df_filled.isna().sum().sum()
        gaps_filled_advanced = nan_before_advanced - nan_after_advanced
        
        # Count filled channels
        filled_channels = 0
        for col in pos_cols:
            if col in df.columns and col in df_filled.columns:
                orig_nans = np.sum(np.isnan(df[col].values))
                filled_nans = np.sum(np.isnan(df_filled[col].values))
                if filled_nans < orig_nans:
                    filled_channels += 1
        
        print(f"   ✅ Gaps filled in {filled_channels} position channels")
        print(f"🔍 AUDIT: Advanced gap filling - NaNs before: {nan_before_advanced}, after: {nan_after_advanced}, filled: {gaps_filled_advanced}")
        
    except Exception as e:
        print(f"   ⚠️  Gap filling failed: {e}")
        print("   Continuing with artifact-truncated data only...")
    
    # Apply bounded SLERP for quaternion gaps (if any)
    print("🔄 Applying BOUNDED SLERP interpolation with hemisphere continuity to segments...")
    quat_cols = [c for c in df_filled.columns if c.endswith(("__qx", "__qy", "__qz", "__qw"))]
    segments = set(c.split("__")[0] for c in quat_cols)
    
    for seg in segments:
        seg_cols = [f"{seg}__qx", f"{seg}__qy", f"{seg}__qz", f"{seg}__qw"]
        if all(col in df_filled.columns for col in seg_cols):
            try:
                # Simple linear interpolation for quaternions (will be re-normalized)
                for col in seg_cols:
                    df_filled[col] = df_filled[col].interpolate(method="linear", limit=10, limit_area='inside')
            except:
                continue
    
    print("   ✅ Bounded SLERP interpolation completed for all segments")
    
    # Surgical quaternion re-normalization (interpolated frames only)
    print("🔒 Surgical quaternion re-normalization (interpolated frames only)...")
    for seg in segments:
        seg_cols = [f"{seg}__qx", f"{seg}__qy", f"{seg}__qz", f"{seg}__qw"]
        if all(col in df_filled.columns for col in seg_cols):
            try:
                qx, qy = df_filled[f"{seg}__qx"], df_filled[f"{seg}__qy"]
                qz, qw = df_filled[f"{seg}__qz"], df_filled[f"{seg}__qw"]
                
                # Find interpolated frames (where any quaternion was NaN and is now filled)
                orig_qx = df[f"{seg}__qx"]
                interp_mask = orig_qx.isna() & df_filled[f"{seg}__qx"].notna()
                
                if interp_mask.any():
                    # Re-normalize only interpolated frames
                    norms = np.sqrt(qx**2 + qy**2 + qz**2 + qw**2)
                    norms[norms == 0] = 1.0
                    
                    df_filled.loc[interp_mask, f"{seg}__qx"] /= norms[interp_mask]
                    df_filled.loc[interp_mask, f"{seg}__qy"] /= norms[interp_mask]
                    df_filled.loc[interp_mask, f"{seg}__qz"] /= norms[interp_mask]
                    df_filled.loc[interp_mask, f"{seg}__qw"] /= norms[interp_mask]
            except:
                continue
    
    print("   ✅ Surgical re-normalization complete (original frames untouched)")
    
    return df_filled

# Execute the advanced gap filling
fs_target = CONFIG.get("fs_target", 120.0)
print(f"Running High-Fidelity Gap Filling (MAD={MAD_MULTIPLIER}x, Max Gap={MAX_GAP_S}s)...")

# Check if we actually need advanced gap filling
current_nans = df_preprocessed.isna().sum().sum()
if current_nans == 0:
    print("ℹ️  No gaps detected in current data. Skipping advanced gap filling.")
    print("   Data is already clean from basic interpolation.")
else:
    df_preprocessed = advanced_gap_filling(df_preprocessed, fs_target)

# Check remaining NaNs and detailed statistics
remaining_nans = df_preprocessed.isna().sum().sum()
print("✅ High-Fidelity Gap Filling Complete.")
print(f"📊 Quality Control: Remaining NaNs (Critical Gaps > {MAX_GAP_S}s): {remaining_nans}")

# DETAILED POSITION MISSING STATISTICS
pos_cols = [c for c in df_preprocessed.columns if c.endswith(("__px", "__py", "__pz"))]
print("\n📈 Position Missing Data Analysis:")
print(f"Mean missing: {df_preprocessed[pos_cols].isna().mean().mean():.4f}")
print(f"Max missing: {df_preprocessed[pos_cols].isna().mean().max():.4f}")
print("Missing data summary:")
print(df_preprocessed[pos_cols].isna().mean().describe())

Running High-Fidelity Gap Filling (MAD=6.0x, Max Gap=0.1s)...
ℹ️  No gaps detected in current data. Skipping advanced gap filling.
   Data is already clean from basic interpolation.
✅ High-Fidelity Gap Filling Complete.
📊 Quality Control: Remaining NaNs (Critical Gaps > 0.1s): 0

📈 Position Missing Data Analysis:
Mean missing: 0.0000
Max missing: 0.0000
Missing data summary:
count    81.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64


In [7]:
# --- QC Stage: Bone Length Check (Fail Fast) ---
# SCIENTIFIC RATIONALE: Skurowski (2021) identifies bone length consistency 
# as the primary metric for motion capture data quality. High CV% indicates 
# marker occlusion or reconstruction artifacts.

def run_bone_length_qc(df, hierarchy, cfg):
    """
    Quality Gate: Validates the Rigid Body Assumption.
    If SUBJECT_HEIGHT is missing, these mean lengths serve as the 
    internal reference for scaling (Hof, 1996).
    """
    print(f"\n{'='*20} BONE LENGTH QC (Scientific Validation) {'='*20}")
    
    # Thresholds: 2% for Warning, 5% for Critical failure (Skurowski standard)
    thresh_warn = cfg['THRESH'].get('BONE_CV_WARN', 0.02)   
    thresh_alert = cfg['THRESH'].get('BONE_CV_ALERT', 0.05) 
    
    results = []
    
    for child_name, info in hierarchy.items():
        parent_name = info['parent']
        if parent_name is None: continue # Skip Root
            
        try:
            # NB02 standard naming convention: Joint__px
            c_pos = df[[f"{child_name}__px", f"{child_name}__py", f"{child_name}__pz"]].values
            p_pos = df[[f"{parent_name}__px", f"{parent_name}__py", f"{parent_name}__pz"]].values
            
            # Distance calculation
            lengths = np.linalg.norm(c_pos - p_pos, axis=1)
            mean_l = np.nanmean(lengths)
            std_l = np.nanstd(lengths)
            cv = std_l / mean_l if mean_l > 0 else 0.0
            
            status = "PASS"
            if cv > thresh_alert: status = "FAIL 🔴"
            elif cv > thresh_warn: status = "WARN 🟡"
            
            results.append({
                "Bone": f"{parent_name}->{child_name}",
                "Mean_mm": round(mean_l * 1000, 1), # Conversion to mm for biomechanical clarity
                "CV%": round(cv * 100, 2),
                "Status": status
            })
        except KeyError:
            continue

    df_qc = pd.DataFrame(results).sort_values("CV%", ascending=False)
    
    # Summary reporting
    n_fails = sum(df_qc['Status'].str.contains("FAIL"))
    print(f"Checked {len(df_qc)} bones.")
    
    if n_fails > 0:
        print(f"⛔ SCIENTIFIC ALERT: {n_fails} bones exceed the 5% instability threshold.")
    else:
        print("✅ SUCCESS: Rigid body integrity confirmed. Ready for Kinematic Derivatives.")
        
    return df_qc

# --- EXECUTE ---
# --- UPDATED EXECUTION FOR BONE QC ---
# We force display of the results for scientific reporting purposes.

# 1. Run the QC
df_bone_qc = run_bone_length_qc(df_preprocessed, kinematics_map, CONFIG)

# 2. Detailed Print (For the 'Methods' section of your research)
print(f"\n{'='*25} DETAILED SEGMENT ANALYSIS {'='*25}")
print(f"{'Bone Segment':<30} | {'Mean (mm)':<10} | {'CV (%)':<8} | {'Status'}")
print("-" * 65)

for _, row in df_bone_qc.iterrows():
    # Adding a visual marker for very high precision (CV < 1%)
    precision_star = "⭐" if row['CV%'] < 1.0 else "  "
    print(f"{row['Bone']:<30} | {row['Mean_mm']:<10} | {row['CV%']:<8} | {row['Status']} {precision_star}")

# 3. Scientific Metadata for Master Report
mean_overall_cv = df_bone_qc['CV%'].mean()
print("-" * 65)
print(f"📊 SCIENTIFIC SUMMARY: Mean Segment CV across all bones: {mean_overall_cv:.2f}%")
print(f"RATIONALE: A mean CV below 2% indicates high-fidelity tracking (Skurowski, 2021).")


==================== BONE LENGTH QC (Scientific Validation) ====================
Checked 26 bones.
✅ SUCCESS: Rigid body integrity confirmed. Ready for Kinematic Derivatives.

========================= DETAILED SEGMENT ANALYSIS =========================
Bone Segment                   | Mean (mm)  | CV (%)   | Status
-----------------------------------------------------------------
Hips->Spine                    | 80827.4    | 4.89     | WARN 🟡   
Neck->Head                     | 137793.9   | 2.37     | WARN 🟡   
Spine->Spine1                  | 213042.1   | 1.85     | PASS   
Spine1->Neck                   | 220539.0   | 1.49     | PASS   
LeftForeArm->LeftHand          | 233159.2   | 0.0      | PASS ⭐
LeftHandIndex1->LeftHandIndex2 | 43252.0    | 0.0      | PASS ⭐
LeftHand->LeftHandIndex1       | 104650.3   | 0.0      | PASS ⭐
LeftHandThumb2->LeftHandThumb3 | 24567.1    | 0.0      | PASS ⭐
LeftHandThumb1->LeftHandThumb2 | 28743.3    | 0.0      | PASS ⭐
LeftHand->LeftHandThumb1       

In [8]:
# --- CELL 08: Scientific Data Persistence ---
# RATIONALE: Using Parquet format preserves double-precision accuracy, 
# which is critical for reducing noise in kinematic derivatives (NB 04/06).
# The Kinematics Map acts as the 'Scientific Contract' between pipeline stages.

# 1. Save Processed Data (High-Precision Parquet)
out_parquet_path = os.path.join(DERIV_02, f"{RUN_ID}__preprocessed.parquet")
df_preprocessed.to_parquet(out_parquet_path, index=False)

# 2. Save Kinematics Map (JSON)
# RATIONALE: This ensures that NB 06 (Rotation) and NB 08 (CoM) use 
# the exact same skeletal hierarchy validated in this notebook.
out_map_path = os.path.join(DERIV_02, f"{RUN_ID}__kinematics_map.json")
with open(out_map_path, 'w') as f:
    json.dump(kinematics_map, f, indent=4)

print(f"\n✅ PERSISTENCE SUCCESS!")
print(f"📊 Kinematic Data: {out_parquet_path}")
print(f"🧬 Kinematics Map: {out_map_path}")
print("\nProceeding to Notebook 03 (Resample).")


✅ PERSISTENCE SUCCESS!
📊 Kinematic Data: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_02_preprocess\734_T1_P1_R1_Take 2025-12-01 02.18.27 PM__preprocessed.parquet
🧬 Kinematics Map: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_02_preprocess\734_T1_P1_R1_Take 2025-12-01 02.18.27 PM__kinematics_map.json

Proceeding to Notebook 03 (Resample).


In [9]:
# --- CELL 09: FINAL CELL - Export Preprocessing Summary for Master Report ---

def export_preprocess_summary(df_pre, df_post, df_bone_qc, run_id, save_dir, cfg):
    """
    Creates a comprehensive QC JSON report. 
    Essential for Methodological Traceability (Winter, 2009).
    """
    total_cells = df_pre.size
    total_nans_pre = df_pre.isna().sum().sum()
    total_nans_post = df_post.isna().sum().sum()
    
    # Bone QC Metrics (Scientific Integrity)
    mean_cv = df_bone_qc['CV%'].mean() if not df_bone_qc.empty else 100.0
    # Capture bones that exceeded the safety threshold
    alerts = df_bone_qc[df_bone_qc['Status'].str.contains("FAIL|WARN")]['Bone'].tolist()
    
    summary = {
        "run_id": run_id,
        "raw_missing_percent": round((total_nans_pre / total_cells) * 100, 3),
        "post_missing_percent": round((total_nans_post / total_cells) * 100, 3),
        "max_interpolation_gap": cfg.get('MAX_GAP_SIZE', 10),
        "bone_qc_mean_cv": round(mean_cv, 3),
        "bone_qc_status": "GOLD" if mean_cv < 1.0 else "SILVER" if mean_cv < 5.0 else "REJECT",
        "bone_qc_alerts": alerts,
        "worst_bone": df_bone_qc.iloc[0]['Bone'] if not df_bone_qc.empty else "None",
        "interpolation_method": "linear_quaternion_normalized"
    }
    
    # Save to JSON
    out_path = os.path.join(save_dir, f"{run_id}__preprocess_summary.json")
    with open(out_path, 'w') as f:
        json.dump(summary, f, indent=4)
    
    print(f"\n{'='*20} PREPROCESS SUMMARY EXPORTED {'='*20}")
    print(f"✅ Summary Path: {out_path}")
    print(f"📊 Bone QC Mean CV: {summary['bone_qc_mean_cv']}% (Status: {summary['bone_qc_status']})")
    print(f"📉 Missing Data: {summary['raw_missing_percent']}% -> {summary['post_missing_percent']}%")
    print(f"{'='*50}\n")

# --- EXECUTE ---
export_preprocess_summary(df_raw, df_preprocessed, df_bone_qc, RUN_ID, DERIV_02, CONFIG)


==================== PREPROCESS SUMMARY EXPORTED ====================
✅ Summary Path: c:\Users\drorh\OneDrive - Mobileye\Desktop\gaga\derivatives\step_02_preprocess\734_T1_P1_R1_Take 2025-12-01 02.18.27 PM__preprocess_summary.json
📊 Bone QC Mean CV: 0.408% (Status: GOLD)
📉 Missing Data: 0.0% -> 0.0%

